# Multi-Agent Collaboration - DSL query use case

In [129]:
import logging
import boto3
import os
import json
import time
import zipfile
import subprocess
from textwrap import dedent

### Configure Logging

In [130]:
# -----------------------------------------------------------------------------
# Configure Logging
# -----------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

### Import Bedrock Agent utilities

In [131]:
from src.utils.bedrock_agent import Agent, SupervisorAgent, agents_helper, region, account_id

### Define AWS clients

In [132]:
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region_name=region)
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
iam_client = boto3.client('iam', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)

logger.info(f"Region: {region}")
logger.info(f"Account ID: {account_id}")
logger.info(f"Agent Suffix: {agent_suffix}")


2025-02-08 09:35:25,502 - __main__ - INFO - Region: us-west-2
2025-02-08 09:35:25,503 - __main__ - INFO - Account ID: 533267284022
2025-02-08 09:35:25,503 - __main__ - INFO - Agent Suffix: us-west-2-533


### Helper Functions

In [133]:
def create_iam_role(role_name: str) -> str:
    """
    Creates or retrieves an IAM Role with the necessary trust policy for Lambda.
    Attaches AWSLambdaBasicExecutionRole, and adds inline policies for OpenSearch 
    and AOSS access.

    :param role_name: Name of the IAM Role to create or retrieve.
    :return: ARN of the created or retrieved IAM Role.
    """
    logger.info(f"Creating or retrieving IAM Role: {role_name}")
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    try:
        role = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
        )
        logger.info(f"IAM Role {role_name} created.")
    except iam_client.exceptions.EntityAlreadyExistsException:
        logger.info(f"IAM Role {role_name} already exists. Retrieving existing role.")
        role = iam_client.get_role(RoleName=role_name)

    # Attach AWS Lambda execution policy
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
    )
    logger.info(f"Attached AWSLambdaBasicExecutionRole to {role_name}.")

    # Attach additional policies for OpenSearch access
    opensearch_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "es:Describe*",
                    "es:List*",
                    "es:Get*"
                ],
                "Resource": "*"
            }
        ]
    }
    opensearch_policy_name = f"{role_name}-OpenSearchPolicy"
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=opensearch_policy_name,
            PolicyDocument=json.dumps(opensearch_policy_document)
        )
        logger.info(f"Attached OpenSearch policy to IAM Role {role_name}.")
    except Exception as e:
        logger.error(f"Failed to attach OpenSearch policy to IAM Role {role_name}: {str(e)}")

    # Attach the new policy for aoss:APICall
    aoss_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "aoss:*"
                ],
                "Resource": "*"
            }
        ]
    }
    aoss_policy_name = f"{role_name}-AOSSPolicy"
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=aoss_policy_name,
            PolicyDocument=json.dumps(aoss_policy_document)
        )
        logger.info(f"Attached AOSS policy to IAM Role {role_name}.")
    except Exception as e:
        logger.error(f"Failed to attach AOSS policy to IAM Role {role_name}: {str(e)}")

    role_arn = role['Role']['Arn']

    # Wait for IAM role to propagate
    logger.info("Waiting 10 seconds for IAM role to propagate...")
    time.sleep(10)

    return role_arn


def create_lambda_package(source_file: str, zip_file_path: str, dependencies: list):
    """
    Packages a Lambda function and its dependencies into a single ZIP file.

    :param source_file: Path to the Lambda function source code.
    :param zip_file_path: Path to the ZIP file that will be created.
    :param dependencies: A list of Python packages required by the Lambda.
    """
    logger.info(f"Packaging Lambda function from {source_file} with dependencies {dependencies}")
    package_dir = "package"

    # Install dependencies to a local folder
    if not os.path.exists(package_dir):
        os.makedirs(package_dir)
    logger.info("Installing dependencies locally...")
    subprocess.run(
        f"pip install {' '.join(dependencies)} -t {package_dir}",
        shell=True,
        check=True
    )

    # Create ZIP file with dependencies and function
    logger.info(f"Creating Lambda deployment package: {zip_file_path}")
    with zipfile.ZipFile(zip_file_path, 'w') as zipf:
        # Add dependencies
        for root, _, files in os.walk(package_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, package_dir)
                zipf.write(file_path, arcname)

        # Add the Lambda function code
        zipf.write(source_file, os.path.basename(source_file))

    # Cleanup temporary package directory
    logger.info("Cleaning up temporary package directory...")
    subprocess.run(f"rm -rf {package_dir}", shell=True)
    logger.info("Lambda package created successfully.")


def create_lambda_function(function_name: str,
                           role_arn: str,
                           handler: str,
                           runtime: str,
                           zip_file_path: str,
                           region_name: str = region) -> dict:
    """
    Creates or updates an AWS Lambda function.

    :param function_name: Name of the Lambda function to create or update.
    :param role_arn: ARN of the IAM Role that Lambda will assume.
    :param handler: The function handler (e.g., 'index.lambda_handler').
    :param runtime: The Lambda runtime (e.g., 'python3.12').
    :param zip_file_path: Path to the ZIP file containing the Lambda code.
    :param region_name: AWS region where the Lambda will be created.
    :return: The response from the create_function or update_function_code API call.
    """
    logger.info(f"Creating/updating Lambda function: {function_name}")
    lambda_client = boto3.client('lambda', region_name=region_name)

    with open(zip_file_path, 'rb') as f:
        zip_content = f.read()

    try:
        response = lambda_client.create_function(
            FunctionName=function_name,
            Runtime=runtime,
            Role=role_arn,
            Handler=handler,
            Code={'ZipFile': zip_content},
            Description='Lambda function to execute DSL queries',
            Timeout=15,
            MemorySize=128,
            Publish=True
        )
        logger.info(f"Lambda function {function_name} created successfully.")
    except lambda_client.exceptions.ResourceConflictException:
        logger.info(f"Lambda function {function_name} already exists. Updating its code...")
        response = lambda_client.update_function_code(
            FunctionName=function_name,
            ZipFile=zip_content
        )
        logger.info(f"Lambda function {function_name} updated successfully.")

    return response


def add_resource_based_policy(function_name: str,
                              agent_ids: list,
                              region_name: str,
                              account_id: str):
    """
    Adds a resource-based policy to the specified Lambda function to allow invocation
    from one or more Bedrock agents.

    :param function_name: Name of the Lambda function.
    :param agent_ids: List of agent IDs permitted to invoke this Lambda.
    :param region_name: AWS region.
    :param account_id: AWS account ID.
    """
    logger.info(f"Adding resource-based policy to Lambda function {function_name} for agents: {agent_ids}")
    statement_id_prefix = "AllowExecutionFromBedrockAgent"
    policy_doc = {
        "Version": "2012-10-17",
        "Statement": []
    }

    for agent_id in agent_ids:
        sid = f"{statement_id_prefix}_{agent_id}"
        policy_doc['Statement'].append({
            "Sid": sid,
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "lambda:InvokeFunction",
            "Resource": f"arn:aws:lambda:{region_name}:{account_id}:function:{function_name}",
            "Condition": {
                "ArnLike": {
                    "AWS:SourceArn": f"arn:aws:bedrock:{region_name}:{account_id}:agent/{agent_id}"
                }
            }
        })

    # Retrieve existing policy and remove any existing statements with the same prefix
    try:
        existing_policy = lambda_client.get_policy(FunctionName=function_name)
        existing_policy_doc = json.loads(existing_policy['Policy'])
        for stmt in existing_policy_doc['Statement']:
            if stmt['Sid'].startswith(statement_id_prefix):
                sid_to_remove = stmt['Sid']
                logger.info(f"Removing existing statement: {sid_to_remove}")
                lambda_client.remove_permission(
                    FunctionName=function_name,
                    StatementId=sid_to_remove
                )
    except lambda_client.exceptions.ResourceNotFoundException:
        logger.info(f"No existing policy found for Lambda function {function_name}.")
    except Exception as e:
        logger.error(f"Error retrieving/removing existing policy for {function_name}: {str(e)}")

    # Add new permissions
    for stmt in policy_doc['Statement']:
        sid_val = stmt['Sid']
        try:
            lambda_client.add_permission(
                FunctionName=function_name,
                StatementId=sid_val,
                Action=stmt['Action'],
                Principal=stmt['Principal']['Service'],
                SourceArn=stmt['Condition']['ArnLike']['AWS:SourceArn']
            )
            logger.info(f"Added permission for statement: {sid_val}")
        except Exception as e:
            logger.error(f"Failed to add resource-based policy for {function_name}, statement {sid_val}: {str(e)}")


## Main Execution

In [134]:
# Load Shipping Schema 
with open('schemas/ecom_shipping_schema.json', 'r') as file:
    ecom_shipping_schema = json.load(file)
ecom_shipping_schema_string = json.dumps(ecom_shipping_schema, indent=2)

# Agent foundation model 
agent_foundation_model = [
    "anthropic.claude-3-5-sonnet-20241022-v2:0"
]

# Force re-create default setting for Agent objects, but for now set to False
Agent.set_force_recreate_default(True)

In [135]:
"""
Main execution flow:
    1. Create an IAM Role for Lambda.
    2. Create/Update two Lambda functions (execute-dsl-query, execute-modified-dsl-query).
    3. Create DSL Query Agent & Query Fixer Agent referencing those Lambda functions.
    4. Retrieve the newly created agent IDs.
    5. Add resource-based policies to each Lambda function for those agent IDs.
    6. Create the Supervisor Agent to orchestrate both DSL Query and Query Fixer agents.
    7. Invoke the Supervisor Agent with a sample query.
    8. Delete the agents (cleanup).
"""
# -------------------------------------------------------------------------
# 1. Create (or retrieve) IAM Role for Lambda
# -------------------------------------------------------------------------
IAM_ROLE_NAME = f"LambdaExecutionRole-{agent_suffix}"
role_arn = create_iam_role(IAM_ROLE_NAME)


# -------------------------------------------------------------------------
# 2. Create the first Lambda (execute-dsl-query)
# -------------------------------------------------------------------------
DSL_QUERY_LAMBDA_NAME = f"execute-dsl-query-{agent_suffix}"
DSL_QUERY_LAMBDA_PATH = "src/lambda/execute_dsl_query.py"
DSL_QUERY_ZIP_PATH = "dsl_query_function.zip"

if not os.path.exists(DSL_QUERY_LAMBDA_PATH):
    logger.error(f"Error: {DSL_QUERY_LAMBDA_PATH} does not exist.")

DEPENDENCIES = ["opensearch-py", "requests", "urllib3"]

# Package & create the Lambda
create_lambda_package(DSL_QUERY_LAMBDA_PATH, DSL_QUERY_ZIP_PATH, DEPENDENCIES)
create_lambda_function(
    function_name=DSL_QUERY_LAMBDA_NAME,
    role_arn=role_arn,
    handler="execute_dsl_query.lambda_handler",
    runtime="python3.12",
    zip_file_path=DSL_QUERY_ZIP_PATH
)
os.remove(DSL_QUERY_ZIP_PATH)


2025-02-08 09:35:25,532 - __main__ - INFO - Creating or retrieving IAM Role: LambdaExecutionRole-us-west-2-533
2025-02-08 09:35:25,882 - __main__ - INFO - IAM Role LambdaExecutionRole-us-west-2-533 already exists. Retrieving existing role.
2025-02-08 09:35:26,135 - __main__ - INFO - Attached AWSLambdaBasicExecutionRole to LambdaExecutionRole-us-west-2-533.
2025-02-08 09:35:26,268 - __main__ - INFO - Attached OpenSearch policy to IAM Role LambdaExecutionRole-us-west-2-533.
2025-02-08 09:35:26,401 - __main__ - INFO - Attached AOSS policy to IAM Role LambdaExecutionRole-us-west-2-533.
2025-02-08 09:35:26,403 - __main__ - INFO - Waiting 10 seconds for IAM role to propagate...
2025-02-08 09:35:36,406 - __main__ - INFO - Packaging Lambda function from src/lambda/execute_dsl_query.py with dependencies ['opensearch-py', 'requests', 'urllib3']
2025-02-08 09:35:36,409 - __main__ - INFO - Installing dependencies locally...


  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl (196 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cach

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.16.1 requires botocore<1.35.89,>=1.35.74, but you have botocore 1.36.9 which is incompatible.
datasets 2.21.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
awscli 1.34.8 requires botocore==1.35.8, but you have botocore 1.36.9 which is incompatible.
awscli 1.34.8 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.11.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
2025-02-08 09:35:37,567 - __main__ - INFO - Creating Lambda deployment package: dsl_query_function.zip
2025-02-08 09:35:37,609 - __main__ - INFO - Cleaning up temporary package directory...
2025-02-08 09

2025-02-08 09:35:40,138 - __main__ - INFO - Lambda function execute-dsl-query-us-west-2-533 already exists. Updating its code...
2025-02-08 09:35:43,004 - __main__ - INFO - Lambda function execute-dsl-query-us-west-2-533 updated successfully.


In [136]:
# -------------------------------------------------------------------------
# 2(b). Create the second Lambda (execute-modified-dsl-query)
# -------------------------------------------------------------------------
MODIFIED_QUERY_LAMBDA_NAME = f"execute-modified-dsl-query-{agent_suffix}"
MODIFIED_QUERY_LAMBDA_PATH = "src/lambda/execute_modified_dsl_query.py"
MODIFIED_QUERY_ZIP_PATH = "modified_query_function.zip"

if not os.path.exists(MODIFIED_QUERY_LAMBDA_PATH):
    logger.error(f"Error: {MODIFIED_QUERY_LAMBDA_PATH} does not exist.")

create_lambda_package(MODIFIED_QUERY_LAMBDA_PATH, MODIFIED_QUERY_ZIP_PATH, DEPENDENCIES)
create_lambda_function(
    function_name=MODIFIED_QUERY_LAMBDA_NAME,
    role_arn=role_arn,
    handler="execute_modified_dsl_query.lambda_handler",
    runtime="python3.12",
    zip_file_path=MODIFIED_QUERY_ZIP_PATH
)
os.remove(MODIFIED_QUERY_ZIP_PATH)

2025-02-08 09:35:43,021 - __main__ - INFO - Packaging Lambda function from src/lambda/execute_modified_dsl_query.py with dependencies ['opensearch-py', 'requests', 'urllib3']
2025-02-08 09:35:43,022 - __main__ - INFO - Installing dependencies locally...


  Using cached opensearch_py-2.8.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached opensearch_py-2.8.0-py3-none-any.whl (353 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl (196 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cach

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.16.1 requires botocore<1.35.89,>=1.35.74, but you have botocore 1.36.9 which is incompatible.
datasets 2.21.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 2.21.0 requires fsspec[http]<=2024.6.1,>=2023.1.0, but you have fsspec 2024.12.0 which is incompatible.
awscli 1.34.8 requires botocore==1.35.8, but you have botocore 1.36.9 which is incompatible.
awscli 1.34.8 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.11.2 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
2025-02-08 09:35:43,963 - __main__ - INFO - Creating Lambda deployment package: modified_query_function.zip
2025-02-08 09:35:44,002 - __main__ - INFO - Cleaning up temporary package directory...
2025-02-

2025-02-08 09:35:46,517 - __main__ - INFO - Lambda function execute-modified-dsl-query-us-west-2-533 already exists. Updating its code...
2025-02-08 09:35:49,970 - __main__ - INFO - Lambda function execute-modified-dsl-query-us-west-2-533 updated successfully.


In [137]:
# -------------------------------------------------------------------------
# 1. Create the DSL Query Agent
# -------------------------------------------------------------------------
dsl_query_lambda_arn = f"arn:aws:lambda:{region}:{account_id}:function:{DSL_QUERY_LAMBDA_NAME}"

logger.info("Creating DSL Query Agent...")
dsl_query_agent = Agent.direct_create(
    name=f"dsl-query-agent-{agent_suffix}",
    role="DSL Query Creator",
    goal="Convert natural language queries into Elasticsearch Query DSL. When addressing the user, use the user name. The user's name is in $prompt_session_attributes$.",
    instructions=f"""
    Generate Query DSL for Elasticsearch.

    Rules:
    - Use the e-commerce shipping schema.
    - Output only within <json>...</json> tags.
    - Follow Query DSL syntax strictly.

    Query Tips:
    - For keyword fields: use term, prefix, or wildcard.
    - For text fields: use match.
    - For nested fields: use nested queries.
    - For date fields: use range with date math.
    - For aggregations: use terms for counting.

    Schema:
    {ecom_shipping_schema_string}

    
    """,
    tool_code=dsl_query_lambda_arn,
    tool_defs=[
        {
            "name": "execute_dsl_query",
            "description": "Executes a DSL query and returns results",
            "parameters": {
                "dsl_query": {
                    "description": "The DSL query to execute",
                    "type": "string",
                    "required": True,
                }
            }
        }
    ]
)

# -------------------------------------------------------------------------
# 2. Create the Query Fixer Agent
# -------------------------------------------------------------------------
modified_query_lambda_arn = f"arn:aws:lambda:{region}:{account_id}:function:{MODIFIED_QUERY_LAMBDA_NAME}"

logger.info("Creating Query Fixer Agent...")
query_fixer_agent = Agent.direct_create(
    name=f"query-fixer-agent-{agent_suffix}",
    role="Query Repair Specialist",
    goal="Fix and optimize failed DSL queries",
    instructions=f"""
    Debug and optimize DSL queries.

    Tasks:
    1. Diagnose errors from OpenSearch messages.
    2. Fix field mapping issues.
    3. Relax strict queries with wildcards or wider date ranges.
    4. Keep original intent while optimizing.
    5. Identify schema gaps and suggest fixes.

    Output Format:
    - Modified query in <json> tags.
    - Revision notes in <notes> tags.

    Schema:
    {ecom_shipping_schema_string}
    """,
    tool_code=modified_query_lambda_arn,
    tool_defs=[
        {
            "name": "retry_query",
            "description": "Retries a modified version of the failed query",
            "parameters": {
                "modified_dsl_query": {
                    "description": "The corrected DSL query",
                    "type": "string",
                    "required": True
                },
                "revision_notes": {
                    "description": "Description of modifications made",
                    "type": "string",
                },
            }
        }
    ]
)

# -------------------------------------------------------------------------
# 3. Create the Knowledge Base (KB) Response Agent
# -------------------------------------------------------------------------
logger.info("Creating KB Response Agent...")
kb_rag_agent = Agent.direct_create(
    name=f"kb-response-agent-{agent_suffix}",
    role="Knowledge Base Content Analyzer",
    goal="Analyze documents and generate fact-based responses",
    instructions="""
    Analyze documents and provide clear, factual responses.

    Response Structure:
    1. Direct answer.
    2. Supporting evidence with quotes.
    3. Source citations like [doc_id:para_num].
    4. Confidence level: High, Medium, or Low.
    """,
    kb_descr="Use the knowledge base to extract relevant information and create accurate responses.",
    kb_id="5GADU65GNF",
    verbose=True
)

2025-02-08 09:35:49,988 - __main__ - INFO - Creating DSL Query Agent...



Deleting existing agent and corresponding lambda for: dsl-query-agent-us-west-2-533...
Agent dsl-query-agent-us-west-2-533 not found
Creating agent dsl-query-agent-us-west-2-533...
Created agent, id: XIM19QMBIF, alias id: TSTALIASID

Adding action group with Lambda: arn:aws:lambda:us-west-2:533267284022:function:execute-dsl-query-us-west-2-533...
Waiting for agent status to change. Current status CREATING
Agent id XIM19QMBIF current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id XIM19QMBIF current status: PREPARED


2025-02-08 09:36:18,437 - __main__ - INFO - Creating Query Fixer Agent...


DONE: Agent: dsl-query-agent-us-west-2-533, id: XIM19QMBIF, alias id: R2HJKYCZ7Y


Deleting existing agent and corresponding lambda for: query-fixer-agent-us-west-2-533...
Agent query-fixer-agent-us-west-2-533 not found
Creating agent query-fixer-agent-us-west-2-533...
Created agent, id: 1DOGR8JVAN, alias id: TSTALIASID

Adding action group with Lambda: arn:aws:lambda:us-west-2:533267284022:function:execute-modified-dsl-query-us-west-2-533...
Waiting for agent status to change. Current status CREATING
Agent id 1DOGR8JVAN current status: NOT_PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id 1DOGR8JVAN current status: PREPARED


2025-02-08 09:36:46,783 - __main__ - INFO - Creating KB Response Agent...


DONE: Agent: query-fixer-agent-us-west-2-533, id: 1DOGR8JVAN, alias id: NZUC32XTSD


Deleting existing agent and corresponding lambda for: kb-response-agent-us-west-2-533...
Agent kb-response-agent-us-west-2-533 not found
Creating agent kb-response-agent-us-west-2-533...
Creating agent: kb-response-agent-us-west-2-533...
Created agent IAM role: arn:aws:iam::533267284022:role/DEFAULT_AgentExecutionRole...
Creating agent: kb-response-agent-us-west-2-533 with model: us.anthropic.claude-3-5-sonnet-20240620-v1:0...
kwargs: {}
Created agent, resulting id: FFQ7TANJDK
Created agent, id: FFQ7TANJDK, alias id: TSTALIASID

Waiting for agent status to change. Current status CREATING
Agent id FFQ7TANJDK current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id FFQ7TANJDK current status: PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id FFQ7TANJDK current status: PREPARED
DONE: Agent: kb-response-agent-us-west-2-533, id: FFQ7TANJ

In [138]:
# -------------------------------------------------------------------------
# 4. Retrieve the newly created Agent IDs
# -------------------------------------------------------------------------
logger.info("Retrieving DSL Query Agent ID...")
dsl_query_agent_id = agents_helper.get_agent_id_by_name(dsl_query_agent.name)
logger.info(f"DSL Query Agent ID: {dsl_query_agent_id}")

logger.info("Retrieving Query Fixer Agent ID...")
query_fixer_agent_id = agents_helper.get_agent_id_by_name(query_fixer_agent.name)
logger.info(f"Query Fixer Agent ID: {query_fixer_agent_id}")

logger.info("Retrieving KB Response Agent ID...")
kb_rag_agent_id = agents_helper.get_agent_id_by_name(kb_rag_agent.name)
logger.info(f"KB Response Agent ID: {kb_rag_agent_id}")


2025-02-08 09:37:20,607 - __main__ - INFO - Retrieving DSL Query Agent ID...
2025-02-08 09:37:20,686 - __main__ - INFO - DSL Query Agent ID: XIM19QMBIF
2025-02-08 09:37:20,686 - __main__ - INFO - Retrieving Query Fixer Agent ID...
2025-02-08 09:37:20,762 - __main__ - INFO - Query Fixer Agent ID: 1DOGR8JVAN
2025-02-08 09:37:20,763 - __main__ - INFO - Retrieving KB Response Agent ID...
2025-02-08 09:37:20,842 - __main__ - INFO - KB Response Agent ID: FFQ7TANJDK


In [139]:
# -------------------------------------------------------------------------
# 5. Add resource-based policy to each Lambda so the Agents can invoke them
# -------------------------------------------------------------------------
add_resource_based_policy(DSL_QUERY_LAMBDA_NAME, [dsl_query_agent_id], region, account_id)
add_resource_based_policy(MODIFIED_QUERY_LAMBDA_NAME, [query_fixer_agent_id], region, account_id)

2025-02-08 09:37:20,852 - __main__ - INFO - Adding resource-based policy to Lambda function execute-dsl-query-us-west-2-533 for agents: ['XIM19QMBIF']
2025-02-08 09:37:20,993 - __main__ - INFO - Removing existing statement: AllowExecutionFromBedrockAgent_DFJH3ASDJV
2025-02-08 09:37:21,162 - __main__ - INFO - Added permission for statement: AllowExecutionFromBedrockAgent_XIM19QMBIF
2025-02-08 09:37:21,165 - __main__ - INFO - Adding resource-based policy to Lambda function execute-modified-dsl-query-us-west-2-533 for agents: ['1DOGR8JVAN']
2025-02-08 09:37:21,210 - __main__ - INFO - Removing existing statement: AllowExecutionFromBedrockAgent_NWA8IBW9JR
2025-02-08 09:37:21,370 - __main__ - INFO - Added permission for statement: AllowExecutionFromBedrockAgent_1DOGR8JVAN


In [140]:
# -------------------------------------------------------------------------
# 4. Create the Supervisor Agent
# -------------------------------------------------------------------------
logger.info("Creating Supervisor Agent...")
supervisor_agent = SupervisorAgent.direct_create(
    name=f"supervisor-agent-{agent_suffix}",
    role="Query Pipeline Orchestrator",
    collaboration_type="SUPERVISOR",
    collaborator_objects=[dsl_query_agent, query_fixer_agent, kb_rag_agent],
    collaborator_agents=[
        {
            "agent": dsl_query_agent.name,
            "instructions": "Handle structured data queries. DSL Query Agent generates Query DSL.",
            "relay_conversation_history": "DISABLED"
        },
        {
            "agent": query_fixer_agent.name,
            "instructions": "Engage when queries fail or need improvement. Query Fixer applies fixes and optimizations.",
            "relay_conversation_history": "DISABLED"
        },
        {
            "agent": kb_rag_agent.name,
            "instructions": "Handle document analysis queries. KB Agent provides responses with evidence.",
            "relay_conversation_history": "DISABLED"
        }
    ],
    instructions="""
        1. Always address the user by name in responses. User name is $prompt_session_attributes$
        2. Only provide responses if the user's job title is "Data Analyst" — otherwise, deny the request politely. Here's the user's job title: $prompt_session_attributes$
        3. Structured Data Queries: Route to DSL Query Agent.
           - If errors or no results, send to Query Fixer Agent (up to 3 retries).
        4. Content Analysis Queries: Route to KB Response Agent.
        5. Deliver the final, well-structured answer to the user.
    """
)

2025-02-08 09:37:21,384 - __main__ - INFO - Creating Supervisor Agent...


Agent supervisor-agent-us-west-2-533 not found

Created supervisor, id: CDOQLLFCB5, alias id: TSTALIASID

  associating sub-agents / collaborators to supervisor...
Waiting for agent status to change. Current status CREATING
Agent id CDOQLLFCB5 current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id CDOQLLFCB5 current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id CDOQLLFCB5 current status: PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id CDOQLLFCB5 current status: PREPARED
DONE: Agent: supervisor-agent-us-west-2-533, id: CDOQLLFCB5, alias id: WUKUKFHLFF



In [141]:
# Inspect attributes of the SupervisorAgent object
# print(dir(supervisor_agent))


In [142]:
# Retrieve the Supervisor Agent ID
logger.info("Retrieving Supervisor Agent ID...")
supervisor_agent_id = agents_helper.get_agent_id_by_name(supervisor_agent.name)
logger.info(f"Supervisor Agent ID: {supervisor_agent_id}")

2025-02-08 09:37:56,933 - __main__ - INFO - Retrieving Supervisor Agent ID...
2025-02-08 09:37:57,055 - __main__ - INFO - Supervisor Agent ID: CDOQLLFCB5


In [143]:
# Lets get the agent based on the ID
# supervisor_agentV2 = agents_helper.get_agent_by_id("KGTOVCVLKI")

In [144]:
# # -------------------------------------------------------------------------
# # 7. Invoke the Supervisor Agent with a sample query
# # -------------------------------------------------------------------------
# response = supervisor_agent.invoke(
#     input_text="How many orders have been shipped by USPS?",
#     session_id="12345",
#     enable_trace=True,
#     trace_level="core"
# )
# logger.info(f"Supervisor agent response: {response}")

In [145]:
# # -------------------------------------------------------------------------
# # 7. Invoke the Supervisor Agent with a sample query
# # -------------------------------------------------------------------------
# response = supervisor_agent.invoke(
#     input_text="How many orders have been shipped by USPS?",
#     session_id="18345",
#     enable_trace=True,
#     trace_level="all",
#     session_state={
#         "sessionAttributes": {
#             "currentTimestamp": "2022-01-01T12:00:00Z"
#         },
#         "promptSessionAttributes": {
#             "userName": "Alice Mallory",
#             "jobTitle": "Data Analyst",
#         }
#     }
# )
# logger.info(f"Supervisor agent response: {response}")

In [146]:
# -------------------------------------------------------------------------
# 7. Invoke the Supervisor Agent with a sample query
# -------------------------------------------------------------------------
response = supervisor_agent.invoke(
    input_text="What are the priority orders with shipping cost greater than $30?",
    session_id="1835",
    enable_trace=True,
    trace_level="all",
    session_state={
        "sessionAttributes": {
            "currentTimestamp": "2022-01-01T12:00:00Z"
        },
        "promptSessionAttributes": {
            "userName": "Alice Mallory",
            "jobTitle": "Data Analyst",
        }
    }
)
logger.info(f"Supervisor agent response: {response}")

invokeAgent API response object: {'ResponseMetadata': {'RequestId': '28ceca75-a289-431a-b2c2-0714518c94c5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 08 Feb 2025 17:37:57 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '28ceca75-a289-431a-b2c2-0714518c94c5', 'x-amz-bedrock-agent-session-id': '1835', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '1835', 'completion': <botocore.eventstream.EventStream object at 0x116367ad0>}
---
{
  "agentAliasId": "WUKUKFHLFF",
  "agentId": "CDOQLLFCB5",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-west-2:533267284022:agent-alias/CDOQLLFCB5/WUKUKFHLFF"
    }
  ],
  "sessionId": "1835",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
  

2025-02-08 09:38:22,344 - __main__ - INFO - Supervisor agent response: Hello Alice, I found 2 priority orders with shipping costs over $30:

1. ORD345678 - Medical Supplies from MedSupply Corp to Central Hospital, shipping cost: $35.99
2. ORD789123 - iPhone 15 Pro from Tech Solutions Inc to Maria Garcia, shipping cost: $45.99

Both orders are currently active with one in Processing status and the other In Transit.


---
---- Step 4 ----
Took 5.3s, using 2066 tokens (in: 1878, out: 188) to complete prior action, observe, orchestrate.
{
  "agentAliasId": "WUKUKFHLFF",
  "agentId": "CDOQLLFCB5",
  "agentVersion": "1",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-west-2:533267284022:agent-alias/CDOQLLFCB5/WUKUKFHLFF"
    }
  ],
  "sessionId": "1835",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationOutput": {
        "metadata": {
          "usage": {
            "inputTokens": 1878,
            "outputTokens": 188
          }
        },
        "rawResponse": {
          "content": "{\"stop_sequence\":null,\"usage\":{\"input_tokens\":1878,\"output_tokens\":188,\"cache_read_input_tokens\":null,\"cache_creation_input_tokens\":null},\"model\":\"claude-3-5-sonnet-20241022\",\"type\":\"message\",\"id\":\"msg_bdrk_01BxZXsgfU6SMoM5N76SEYDS\",\"content\":[{\"name\":null,\"type\":\"text\",\"id\":null,\"source\":null,\"input\":null,\"is_error\":null,\"text\":\"<thinking>I'l

In [ ]:
# # -------------------------------------------------------------------------
# # 7. Invoke the Supervisor Agent with a sample query
# # -------------------------------------------------------------------------
# response = supervisor_agent.invoke(
#     input_text="How many orders have been shipped by DHL?",
#     session_id="12346",
#     enable_trace=True,
#     trace_level="core",
#     session_state={
#         "promptSessionAttributes": {
#             "userName": "Alice Mallory",
#         }
#     }

# )
# logger.info(f"Supervisor agent response: {response}")

invokeAgent API input parameters: input_text: How many orders have been shipped by DHL?, agent_id: QDMOLJO9TV, agent_alias_id: LXJCIDHVKO, session_id: 12345, session_state: {}, enable_trace: True, end_session: False, trace_level: all, multi_agent_names: {'CSKABIRKLE/V6WM6PSNHT': 'dsl-query-agent-us-west-2-533', 'SGSWZUVNOB/7KNLAUFPQC': 'query-fixer-agent-us-west-2-533', 'PM73T3Y9BR/GHH3JVSG1F': 'kb-response-agent-us-west-2-533', 'QDMOLJO9TV/LXJCIDHVKO': 'supervisor-agent-us-west-2-533'}


In [ ]:
# response = supervisor_agent.invoke(
#     input_text="What are the effects of Covid-19 on e-commerce?",
#     session_id="1245",
#     enable_trace=True,
#     trace_level="core"
# )

In [ ]:
# response = supervisor_agent.invoke(
#     input_text="How many orders have recipients in Spain and were last updated during customs clearance after January 16, 2024?",
#     session_id="1245",
#     enable_trace=True,
#     trace_level="core"
# )

In [ ]:
# response = supervisor_agent.invoke(
#     input_text="What are the temperature controlled packages delivered within 2 hours?",
#     session_id="1245",
#     enable_trace=True,
#     trace_level="core"
# )

In [108]:
# logger.info("Deleting Supervisor Agent...")
# agents_helper.delete_agent(supervisor_agent.name, verbose=True)

In [147]:
# -------------------------------------------------------------------------
# 8. Cleanup: Delete the created agents
# -------------------------------------------------------------------------
logger.info("Deleting Supervisor Agent...")
agents_helper.delete_agent(supervisor_agent.name, verbose=True)

logger.info("Deleting DSL Query Agent...")
agents_helper.delete_agent(dsl_query_agent.name, verbose=True)

logger.info("Deleting Query Fixer Agent...")
agents_helper.delete_agent(query_fixer_agent.name, verbose=True)

logger.info("Deleting KB Response Agent...")
agents_helper.delete_agent(kb_rag_agent.name, verbose=True)

2025-02-08 11:32:12,979 - __main__ - INFO - Deleting Supervisor Agent...


Found target agent, name: supervisor-agent-us-west-2-533, id: CDOQLLFCB5
Deleting aliases for agent CDOQLLFCB5...
Deleting alias TSTALIASID from agent CDOQLLFCB5
Deleting alias WUKUKFHLFF from agent CDOQLLFCB5
Deleting agent: CDOQLLFCB5...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_supervisor-agent-us-west-2-533...


2025-02-08 11:32:24,949 - __main__ - INFO - Deleting DSL Query Agent...


Found target agent, name: dsl-query-agent-us-west-2-533, id: XIM19QMBIF
Deleting aliases for agent XIM19QMBIF...
Deleting alias R2HJKYCZ7Y from agent XIM19QMBIF
Deleting alias TSTALIASID from agent XIM19QMBIF
Deleting agent: XIM19QMBIF...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_dsl-query-agent-us-west-2-533...


2025-02-08 11:32:36,909 - __main__ - INFO - Deleting Query Fixer Agent...


Found target agent, name: query-fixer-agent-us-west-2-533, id: 1DOGR8JVAN
Deleting aliases for agent 1DOGR8JVAN...
Deleting alias NZUC32XTSD from agent 1DOGR8JVAN
Deleting alias TSTALIASID from agent 1DOGR8JVAN
Deleting agent: 1DOGR8JVAN...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_query-fixer-agent-us-west-2-533...


2025-02-08 11:32:48,732 - __main__ - INFO - Deleting KB Response Agent...


Found target agent, name: kb-response-agent-us-west-2-533, id: FFQ7TANJDK
Deleting aliases for agent FFQ7TANJDK...
Deleting alias L4GFLDVI3E from agent FFQ7TANJDK
Deleting alias TSTALIASID from agent FFQ7TANJDK
Deleting agent: FFQ7TANJDK...
Deleting IAM role: AmazonBedrockExecutionRoleForAgents_kb-response-agent-us-west-2-533...
